In [1]:
!pip install bert_score
import bert_score

In [23]:
%%capture
%matplotlib inline
from model_utils import predict_for_test_samples
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import numpy as np
from bert_score import score
from utils import load_obj
from utils import caption_to_gender
from nltk.translate.bleu_score import sentence_bleu

def test(vocab_path, model_path, training_image_ids_path):
    sample_size = 100
    image_folder_path = "./data/images/"
    test_pred_captions = predict_for_test_samples(image_folder_path = image_folder_path,\
                         sample_size = sample_size, vocab_path = vocab_path, model_path = model_path,\
                           training_image_ids_path = training_image_ids_path)

    #load obj
    im_gender_summary = load_obj("im_gender_summary")
    captions_dict = load_obj("captions_dict")

    gt = []
    pred_gender = []
    bleus = []
    for image_id, caption in test_pred_captions.items():
        gt.append(im_gender_summary[image_id]['pred_gt'])
        pred_gender.append(caption_to_gender(caption))
        ref_captions = captions_dict[image_id][0]
        ref_captions = [c.split() for c in ref_captions]
        bleus.append(sentence_bleu(ref_captions, caption.split()))

    #_, _, F1 = score(ref_captions, caption.split())

    return confusion_matrix(gt, pred_gender), accuracy_score(gt, pred_gender), np.mean(bleus)#, F1.mean()

In [26]:
import glob
import re
import numpy as np

results = dict()

model_dirs = glob.glob('./*/testmodels/*/')
for d in model_dirs:
    # get name
    p = re.compile(r'(?<=dels/testmodels/)(.*)(?=_)')
    name = p.findall(d)[0]
    
    # get paths
    vocab_path = d + 'vocab.pkl'
    model_path = d +'best-model.pkl'
    training_image_ids_path = d +'training_image_ids.pkl'
    
    conf_matrix, accuracy, bleu = test(vocab_path = vocab_path, model_path = model_path\
                                             , training_image_ids_path = training_image_ids_path)
    
    results[name] = dict()
    results[name]['confusion_matrix'] = conf_matrix
    results[name]['accuracy'] = accuracy
    results[name]['bleu'] = bleu
    print(f"Evaluated {name}: {results[name]}")

Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab

/home/ubuntu/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use Smoo

Evaluated random_gender_neutral: {'confusion_matrix': array([[ 0, 34,  0],
       [ 0, 34,  0],
       [ 1, 31,  0]]), 'accuracy': 0.34, 'bleu': 1.1478686026431065e-231}
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained 

Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab

Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab

Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab

Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab

Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab

In [28]:
from utils import save_obj

save_obj(results, "model_results")

In [27]:
results

{'random_gender_neutral': {'confusion_matrix': array([[ 0, 34,  0],
         [ 0, 34,  0],
         [ 1, 31,  0]]), 'accuracy': 0.34, 'bleu': 1.1478686026431065e-231},
 'balanced_mode_reg': {'confusion_matrix': array([[21, 13,  0],
         [10, 23,  1],
         [16,  9,  7]]), 'accuracy': 0.51, 'bleu': 1.1162177414255587e-231},
 'balanced_mode_gender_neutral': {'confusion_matrix': array([[34,  0,  0],
         [33,  0,  1],
         [32,  0,  0]]), 'accuracy': 0.34, 'bleu': 1.1566150642509882e-231},
 'balanced_clean_noun_gender_neutral': {'confusion_matrix': array([[24, 10,  0],
         [20, 14,  0],
         [17, 12,  3]]), 'accuracy': 0.41, 'bleu': 1.13234096131087e-231},
 'balanced_clean_noun_reg': {'confusion_matrix': array([[16,  7, 11],
         [ 8, 21,  5],
         [ 5,  4, 23]]), 'accuracy': 0.6, 'bleu': 1.1208756777468883e-231},
 'balanced_clean_reg': {'confusion_matrix': array([[24,  6,  4],
         [13, 16,  5],
         [ 0,  0,  0]]),
  'accuracy': 0.5882352941176471

In [1]:
# from biasanalysis import amp_utils,bias_analysis
from biasanalysis.amp_utils import *
from biasanalysis.bias_analysis import *
from utils import *

def return_bias(train_ids,pred_captions_dict):

    # loading training captions
    train_captions_dict = load_obj('captions_dict')
    train_captions = [caption for id in train_ids for caption in train_captions_dict[id]]
    test_captions = [caption for id in pred_captions_dict for caption in pred_captions_dict[id]]
    bias_amp = bias_amplification(train_captions, test_captions)
    return bias_amp

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import re
from utils import load_obj
import pickle
from model_utils import predict_for_test_samples

bias_models = dict()

model_dirs = glob.glob('./*/testmodels/*/')
for d in model_dirs:
    # get name
    p = re.compile(r'(?<=dels/testmodels/)(.*)(?=_)')
    name = p.findall(d)[0]
    
    image_folder_path = './data/images/'
    sample_size = 100
    vocab_path = d + 'vocab.pkl'
    model_path = d +'best-model.pkl'
    training_image_ids_path = d +'training_image_ids.pkl'
    test_pred_captions = predict_for_test_samples(image_folder_path = image_folder_path,\
                         sample_size = sample_size, vocab_path = vocab_path, model_path = model_path,\
                           training_image_ids_path = training_image_ids_path)
    with open(training_image_ids_path, 'rb') as f:
        train_ids = pickle.load(f)
    
    bias_amp = return_bias(train_ids, test_pred_captions)
    print(bias_amp)
    bias_models[name] = bias_amp
    

Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab

-0.02254791431792559
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrain

-0.024390243902439025
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrai

-0.024390243902439025
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrai

-0.026685393258426966
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrai

...

-0.02254791431792559
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrain

-0.008507621410847216
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrai

-0.020249221183800622
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrai

-0.024719101123595506
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrained model
Loaded vocab file of pretrai

-0.024719101123595506
Loaded vocab file of pretrained model
